In [1]:
import math

def respire_error_estimate(
    sigma: float,
    d: int,
    ds: int,
    dr: int,
    p: int,
    q1: int,
    q2: int,
    q: int,
    nu1: int,
    nu2: int,
    nu3: int,
    t_gsw: int,
    t_auto_expand_regev: int,
    t_auto_expand_gsw: int,
    t_auto_proj: int,
    t_regev_to_gsw: int,
    t_scal_to_vec: int,
    t_compress: int,
    gadget_correction: bool,
    secret_norm_corection: bool,
  ):

  sigma_sq: float = sigma ** 2

  def eta(z: int) -> float:
    s: float = 0
    z_min = -math.floor((z - 1) / 2)
    z_max = math.floor(z / 2)
    assert(len(range(z_min, z_max + 1)) == z)
    for i in range(z_min, z_max + 1):
      s += i ** 2
    return s / z

  def z_ratio(z: int) -> float:
    if gadget_correction:
      return eta(z)
    else:
      return math.floor(z / 2) ** 2

  def t_to_z(t: int) -> int:
    return math.ceil(q ** (1 / t))

  def expand(e_sq: float, t_auto: int, z_auto: int) -> float:
    return 2 * e_sq + t_auto * d * z_ratio(z_auto) * sigma_sq

  def select(e_gsw_sq: float, e_reg_sq: float) -> float:
    return e_reg_sq + 2 * t_gsw * d * z_ratio(z_gsw) * e_gsw_sq

  # def hybrid_mul(e_gsw_sq: float, e_reg_sq: float) -> float:
  #   # TODO fix
  #   # return e_reg_sq + 2 * t_gsw * d * z_ratio(z_gsw) * e_gsw_sq
  #   return 2 * t_gsw * d * z_ratio(z_gsw) * e_gsw_sq

  z_gsw: int = t_to_z(t_gsw)
  z_auto_expand_regev: int = t_to_z(t_auto_expand_regev)
  z_auto_expand_gsw: int = t_to_z(t_auto_expand_gsw)
  z_auto_proj: int = t_to_z(t_auto_proj)
  z_regev_to_gsw: int = t_to_z(t_regev_to_gsw)
  z_scal_to_vec: int = t_to_z(t_scal_to_vec)
  z_compress: int = t_to_z(t_compress)

  #
  # Query expansion
  #

  qi: float = sigma_sq
  q_reg: float = expand(qi, t_auto_expand_gsw, z_auto_expand_gsw)
  q_gsw: float = q_reg
  print("q_reg / q_gsw bits", math.log2(q_reg) / 2)

  n_gsw: int = t_gsw * (nu2 + nu3)

  c_reg: float = q_reg
  for i in range(nu1):
    c_reg = expand(c_reg, t_auto_expand_regev, z_auto_expand_regev)

  print("c_reg bits", math.log2(c_reg) / 2)

  c_reg_prime: float = q_gsw
  for i in range(math.ceil(math.log2(n_gsw))):
    c_reg_prime = expand(c_reg_prime, t_auto_expand_gsw, z_auto_expand_gsw)

  print("c_reg_prime bits", math.log2(c_reg_prime) / 2)

  secret_norm = 1 if secret_norm_corection else 8
  c_gsw = d * c_reg_prime * secret_norm * sigma_sq + 2 * t_regev_to_gsw * d * z_ratio(z_regev_to_gsw) * sigma_sq

  print("c_gsw bits", math.log2(c_gsw) / 2)

  #
  # First dimension
  #

  c_i = (2 ** nu1) * d * (p / 2) * c_reg

  print("c_i bits (first dim)", math.log2(c_i) / 2)

  #
  # Folding
  #

  for b in range(nu2):
    c_i = select(c_gsw, c_i)
    print(b, math.log2(c_i) / 2)

  print("c_i bits (fold)", math.log2(c_i) / 2)

  #
  # Projecting
  #

  for c in range(nu3):
    c_i = select(c_gsw, c_i)
    c_i = expand(c_i, t_auto_proj, z_auto_proj)

  print("c_i bits (proj)", math.log2(c_i) / 2)

  #
  # Scal to Vec
  #
  c = c_i + t_scal_to_vec * d * z_ratio(z_scal_to_vec) * sigma_sq

  print(math.log2(c) / 2)

In [2]:
respire512 = {
  "sigma": 6.4,
  "d": 2048,
  "ds": 512,
  "dr": 512,
  "p": 17,
  "q1": 68,
  "q2": 163841,
  "q": 268369921 * 249561089,
  "nu1": 9,
  "nu2": 9,
  "nu3": 2,
  "t_gsw": 8,
  "t_auto_expand_regev": 8,
  "t_auto_expand_gsw": 16,
  "t_auto_proj": 16,
  "t_regev_to_gsw": 4,
  "t_scal_to_vec": 8,
  "t_compress": 18,
}

respire1024 = {
  "sigma": 6.4,
  "d": 2048,
  "ds": 1024,
  "dr": 256,
  "p": 257,
  "q1": 1028,
  "q2": 4169729,
  "q": 268369921 * 249561089,
  "nu1": 9,
  "nu2": 8,
  "nu3": 3,
  "t_gsw": 8,
  "t_auto_expand_regev": 8,
  "t_auto_expand_gsw": 16,
  "t_auto_proj": 16,
  "t_regev_to_gsw": 4,
  "t_scal_to_vec": 8,
  "t_compress": 18,
}


In [3]:
print("**** Respire512")

print("** Base estimate")
respire_error_estimate(
  **respire512,
  gadget_correction=False,
  secret_norm_corection=False,
)
print()

print("** With gadget correction")
respire_error_estimate(
  **respire512,
  gadget_correction=True,
  secret_norm_corection=False,
)
print()

print("** ... and secret norm correction")
respire_error_estimate(
  **respire512,
  gadget_correction=True,
  secret_norm_corection=True,
)

**** Respire512
** Base estimate
q_reg / q_gsw bits 12.763035628820496
c_reg bits 20.16693513676381
c_reg_prime bits 16.760211738154847
c_gsw bits 26.442061955563396
c_i bits (first dim) 31.710666557388983
0 39.91935012095816
1 40.41934600002251
2 40.71182587673263
3 40.919343939545854
4 41.080307574893496
5 41.21182450307957
6 41.32302051751143
7 41.419342909305314
8 41.50430529555521
c_i bits (fold) 41.50430529555521
c_i bits (proj) 42.61550137539911
42.61550137539911

** With gadget correction
q_reg / q_gsw bits 11.98050655288644
c_reg bits 19.385788816038424
c_reg_prime bits 15.97768146906643
c_gsw bits 25.65948005607034
c_i bits (first dim) 30.929520236663596
0 38.355663155019315
1 38.85565096134258
2 39.14812814709844
3 39.355644864426914
4 39.51660769248128
5 39.64812408247082
6 39.759319712476085
7 39.85564181594976
8 39.94060397795044
c_i bits (fold) 39.94060397795044
c_i bits (proj) 41.05179980150935
41.05179980150935

** ... and secret norm correction
q_reg / q_gsw bits 11.9

In [4]:
print("**** Respire1024")

print("** Base estimate")
respire_error_estimate(
  **respire1024,
  gadget_correction=False,
  secret_norm_corection=False,
)
print()

print("** With gadget correction")
respire_error_estimate(
  **respire1024,
  gadget_correction=True,
  secret_norm_corection=False,
)
print()

print("** ... and secret norm correction")
respire_error_estimate(
  **respire1024,
  gadget_correction=True,
  secret_norm_corection=True,
)

**** Respire1024
** Base estimate
q_reg / q_gsw bits 12.763035628820496
c_reg bits 20.16693513676381
c_reg_prime bits 16.760211738154847
c_gsw bits 26.442061955563396
c_i bits (first dim) 33.66974741136075
0 39.91946646707272
1 40.419404175757805
2 40.71186466115133
3 40.919373028083065
4 41.080330845830396
5 41.21184389558651
6 41.32303713969667
7 41.41935745374132
c_i bits (fold) 41.41935745374132
c_i bits (proj) 43.06205576774218
43.06205576774218

** With gadget correction
q_reg / q_gsw bits 11.98050655288644
c_reg bits 19.385788816038424
c_reg_prime bits 15.97768146906643
c_gsw bits 25.65948005607034
c_i bits (first dim) 32.88860109063536
0 38.35600736202224
1 38.85582308828406
2 39.14824290360284
3 39.35573093375916
4 39.516676548885016
5 39.64818146332836
6 39.75936889638731
7 39.85568485208145
c_i bits (fold) 39.85568485208145
c_i bits (proj) 41.498377690130596
41.498377690130596

** ... and secret norm correction
q_reg / q_gsw bits 11.98050655288644
c_reg bits 19.3857888160384